In [1]:
import pandas as pd
from pymongo import MongoClient
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from pandas.io.json import json_normalize


# Import companies from Mongodb

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


## sort columns from offices

In [3]:
db = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))
df = pd.DataFrame(json_normalize(db))
print(df.shape)
df.head()

<ipython-input-3-8555aed50465>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.DataFrame(json_normalize(db))


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8d,Digg,news,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523


## Droping columns

In [4]:
df_gaming = df
df_gaming.drop(df_gaming[df_gaming.category_code != "games_video"].index, axis=0, inplace=True)
df_gaming = df_gaming.drop(['_id'], axis=1)

In [5]:
df_gaming.dropna(subset = ['offices.city'])
df_gaming.shape
df_gaming["offices.city"].value_counts().head(3)

New York         75
San Francisco    68
                 54
Name: offices.city, dtype: int64

,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
4740,Overstream,games_video,,235 Montgomery St.,Suite 600,94104,San Francisco,CA,USA,37.791391,-122.402859


## Save file as companiesgame_clean

In [6]:
df_gaming.to_json("../companiesgame_clean.json",orient="records")

## Cleaning dataset to get only the design companies

In [7]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [8]:
db = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))
df = pd.DataFrame(json_normalize(db))
print(df.shape)
df.head()
df_design = df
df_design.drop(df_design[df_design.category_code != "design"].index, axis=0, inplace=True)
df_design = df_design.drop(['_id'], axis=1)


<ipython-input-8-175ff8ac3aaf>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.DataFrame(json_normalize(db))


(16705, 12)


In [9]:
df_design.dropna()

,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
2023,99designs,design,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
13928,Graticle,design,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144


## Concat both DataFrames

Getting DataFrame 'result' from 'df_gaming' and 'df_design'

In [10]:
result = pd.concat([df_gaming, df_design], ignore_index=True)

In [11]:
result.shape

(962, 11)

## Saveing file as companies_one

In [12]:
result.to_json("input/companies_one.json",orient="records")

## Tableau result:

[https://public.tableau.com/profile/albert8801#!/vizhome/company_clean/Hoja2?publish=yes]

In [45]:
final_location = df_gaming[df_gaming.name  =="Overstream"]

def finallocation(data):
    return{
        'Address':list(final_location['offices.address1']),
        'Postal Code':list(final_location['offices.zip_code']),
        'City':list(final_location['offices.city']),
        'Latitude':list(final_location['offices.latitude']),
        'Longitude':list(final_location['offices.longitude'])
        
    }



In [46]:
finallocation(final_location)

{'Address': ['235 Montgomery St.'],
 'Postal Code': ['94104'],
 'City': ['San Francisco'],
 'Latitude': [37.791391],
 'Longitude': [-122.402859]}